In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
/FileStore/tables/sales_csv.txt


FileStore/tables/menu_csv.txt


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1615632455331945>:1
----> 1 FileStore/tables/sales_csv.txt()
      4 FileStore/tables/menu_csv.txt

NameError: name 'FileStore' is not defined

In [0]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType,StringType,DateType

schema=StructType([
    StructField("product_id",IntegerType(),True),
    StructField("customer_id",StringType(),True),
    StructField("order_date",DateType(),True),
    StructField("location",StringType(),True),
    StructField("source_order",StringType(),True)
])

sales_df=spark.read.format("csv").option("inferschema","true").schema(schema).load("/FileStore/tables/sales_csv.txt")

display(sales_df)

product_id,customer_id,order_date,location,source_order
1,A,2023-01-01,India,Swiggy
2,A,2022-01-01,India,Swiggy
2,A,2023-01-07,India,Swiggy
3,A,2023-01-10,India,Restaurant
3,A,2022-01-11,India,Swiggy
3,A,2023-01-11,India,Restaurant
2,B,2022-02-01,India,Swiggy
2,B,2023-01-02,India,Swiggy
1,B,2023-01-04,India,Restaurant
1,B,2023-02-11,India,Swiggy


In [0]:
from pyspark.sql.functions import month,year,quarter

sales_df=sales_df.withColumn("order_year",year(sales_df.order_date))
sales_df=sales_df.withColumn("order_month",month(sales_df.order_date))
sales_df=sales_df.withColumn("order_quarter",quarter(sales_df.order_date))
display(sales_df)

product_id,customer_id,order_date,location,source_order,order_year,order_month,order_quarter
1,A,2023-01-01,India,Swiggy,2023,1,1
2,A,2022-01-01,India,Swiggy,2022,1,1
2,A,2023-01-07,India,Swiggy,2023,1,1
3,A,2023-01-10,India,Restaurant,2023,1,1
3,A,2022-01-11,India,Swiggy,2022,1,1
3,A,2023-01-11,India,Restaurant,2023,1,1
2,B,2022-02-01,India,Swiggy,2022,2,1
2,B,2023-01-02,India,Swiggy,2023,1,1
1,B,2023-01-04,India,Restaurant,2023,1,1
1,B,2023-02-11,India,Swiggy,2023,2,1


In [0]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType,StringType,DateType

schema=StructType([
    StructField("product_id",IntegerType(),True),
    StructField("product_name",StringType(),True),
    StructField("price",StringType(),True),
])

menu_df=spark.read.format("csv").option("inferschema","true").schema(schema).load("/FileStore/tables/menu_csv.txt")

display(menu_df)


product_id,product_name,price
1,PIZZA,100
2,Chowmin,150
3,sandwich,120
4,Dosa,110
5,Biryani,80
6,Pasta,180


In [0]:
total_amt_spent=(sales_df.join(menu_df,"product_id")
                 .groupBy("customer_id")
                 .agg({"price":"Sum"})
                 .orderBy("customer_id"))


display(total_amt_spent)

customer_id,sum(price)
A,4260.0
B,4440.0
C,2400.0
D,1200.0
E,2040.0


Databricks visualization. Run in Databricks to view.

In [0]:
total_amt_spent_onfood=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_name")
                        .agg({"price":"Sum"})
                        .orderBy("product_name"))

display(total_amt_spent_onfood)

product_name,sum(price)
Biryani,480.0
Chowmin,3600.0
Dosa,1320.0
PIZZA,2100.0
Pasta,1080.0
sandwich,5760.0


Databricks visualization. Run in Databricks to view.

In [0]:
total_amt_sales_of_month=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_month")
                        .agg({"price":"Sum"}))
                        

display(total_amt_sales_of_month)

order_month,sum(price)
1,2960.0
6,2960.0
3,910.0
5,2960.0
7,910.0
11,910.0
2,2730.0


Databricks visualization. Run in Databricks to view.

In [0]:
total_amt_sales_of_year=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_year")
                        .agg({"price":"Sum"}))

display(total_amt_sales_of_year)

order_year,sum(price)
2023,9990.0
2022,4350.0


Databricks visualization. Run in Databricks to view.

In [0]:
total_amt_sales_of_quarter=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_quarter")
                        .agg({"price":"Sum"}))

display(total_amt_sales_of_quarter)

order_quarter,sum(price)
1,6600.0
3,910.0
4,910.0
2,5920.0


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import count


most_df=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_id","product_name")
                        .agg(count("product_id").alias("product_count"))
                        .orderBy('product_count',ascending=0)
                        .drop('product_id')
                        )               

display(most_df)

product_name,product_count
sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Biryani,6
Pasta,6


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import count


most_df=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_id","product_name")
                        .agg(count("product_id").alias("product_count"))
                        .orderBy('product_count',ascending=0)
                        .drop('product_id')
                        .limit(1)
                        )               

display(most_df)

product_name,product_count
sandwich,48


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import countDistinct


df=(sales_df.filter(sales_df.source_order=='Restaurant')
                        .groupBy("customer_id")
                        .agg(countDistinct("order_date")))         

display(df)

customer_id,count(order_date)
E,5
B,6
D,1
C,3
A,6


Databricks visualization. Run in Databricks to view.

In [0]:
df=(sales_df.join(menu_df,'product_id').groupBy("location").agg({'price':'sum'}))
display(df)

location,sum(price)
India,4860.0
USA,2460.0
UK,7020.0


Databricks visualization. Run in Databricks to view.

In [0]:
df=(sales_df.join(menu_df,'product_id').groupBy("source_order").agg({'price':'sum'}))
display(df)

source_order,sum(price)
zomato,4920.0
Swiggy,6330.0
Restaurant,3090.0


Databricks visualization. Run in Databricks to view.